In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform

# ============================================================
# ДАННЫЕ TSP (как в статье)
# ============================================================

def load_tsp_data_from_paper():
    return np.array([
        [0.41, 0.43],
        [0.18, 0.22],
        [0.82, 0.67],
        [0.66, 0.41],
        [0.55, 0.85],
        [0.27, 0.63],
        [0.73, 0.29],
        [0.58, 0.69],
        [0.86, 0.55],
        [0.32, 0.26],
    ])

def distance_matrix(coords):
    return squareform(pdist(coords))

# ============================================================
# TCNN-ОБНОВЛЕНИЕ
# ============================================================

def tcnn_step(y, k, c, I0, z, alpha, W1, W2, D):
    """
    Один шаг TCNN для TSP (Cheng–Aihara)
    """
    n = D.shape[0]

    # Активация
    y_clip = np.clip(y, -500, 500)
    x = 1.0 / (1.0 + np.exp(-y_clip / c))

    # --- Ограничения (каждый город и позиция — ровно один раз)
    row_sum = np.sum(x, axis=1, keepdims=True)
    col_sum = np.sum(x, axis=0, keepdims=True)
    constraint = W1 * ((row_sum - 1) + (col_sum - 1))

    # --- Градиент длины маршрута
    grad_dist = np.zeros_like(x)
    for i in range(n):
        for j in range(n):
            jp = (j + 1) % n
            jm = (j - 1) % n
            grad_dist[i, j] = W2 * np.sum(
                D[i, :] * (x[:, jp] + x[:, jm])
            )

    # --- Обновление нейронов
    y_new = (
        k * y
        - alpha * constraint
        - alpha * grad_dist
        - z * (x - I0)
    )

    return y_new, x

# ============================================================
# ЭНЕРГИЯ
# ============================================================

def energy_tsp(x, D, W1, W2):
    n = D.shape[0]

    # Ограничения
    row_sum = np.sum(x, axis=1)
    col_sum = np.sum(x, axis=0)
    E1 = W1 / 2 * (np.sum((row_sum - 1)**2) + np.sum((col_sum - 1)**2))

    # Длина маршрута
    E2 = 0.0
    for j in range(n):
        jp = (j + 1) % n
        for i in range(n):
            for k in range(n):
                E2 += D[i, k] * x[i, j] * x[k, jp]
    E2 *= W2

    return E1 + E2

# ============================================================
# ИЗВЛЕЧЕНИЕ МАРШРУТА (winner-takes-all)
# ============================================================

def extract_solution(x):
    n = x.shape[0]
    x_bin = np.zeros_like(x, dtype=int)
    for j in range(n):
        i_star = np.argmax(x[:, j])
        x_bin[i_star, j] = 1
    return x_bin

def is_valid_tour(x_bin):
    return (
        np.all(np.sum(x_bin, axis=0) == 1) and
        np.all(np.sum(x_bin, axis=1) == 1)
    )

# ============================================================
# ЭКСПЕРИМЕНТ
# ============================================================

coords = load_tsp_data_from_paper()
D = distance_matrix(coords)
n = D.shape[0]

# Параметры статьи
k = 0.9
c = 1 / 250
W1 = 1.0
W2 = 1.0
I0 = W1
alpha = 0.015
z0 = 0.08
beta = 0.005

num_steps = 1000
num_trials = 5000
optimal_length = 2.70

global_min = 0
local_min = 0
invalid = 0

for trial in range(num_trials):
    y = np.random.uniform(-1, 1, (n, n))
    z = z0

    best_valid_energy = np.inf
    found = False

    for t in range(num_steps):
        y, x = tcnn_step(y, k, c, I0, z, alpha, W1, W2, D)
        z *= (1 - beta)

        x_bin = extract_solution(x)

        if is_valid_tour(x_bin):
            E = energy_tsp(x_bin.astype(float), D, W1, W2)
            best_valid_energy = E
            found = True

            if z < 1e-3:
                break

    if found:
        if abs(best_valid_energy - optimal_length) < 1e-3:
            global_min += 1
        else:
            local_min += 1
    else:
        invalid += 1

# ==========================================
# ГРАФИК ДИНАМИКИ ЭНЕРГИИ И z(t)
# ==========================================

y = np.random.uniform(-1, 1, (n, n))
z = z0

E_c_vals = []
E_d_vals = []
z_vals = []

for t in range(1000):
    y, x = tcnn_step(y, k, c, I0, z, alpha, W1, W2, D)
    z_vals.append(z)

    E_c_vals.append(energy_tsp(x, D, W1, W2))

    x_bin = extract_solution(x)
    if is_valid_tour(x_bin):
        E_d_vals.append(energy_tsp(x_bin.astype(float), D, W1, W2))
    else:
        E_d_vals.append(E_d_vals[-1] if E_d_vals else 0)

    z *= (1 - beta)

plt.figure(figsize=(10, 6))
plt.plot(E_c_vals, label=r'$E_c(t)$', linewidth=2)
plt.plot(E_d_vals, label=r'$E_D(t)$', linestyle='--')
plt.plot([zz * 10000 for zz in z_vals], label=r'$z(t)\times 10^4$')
plt.xlabel('Iteration')
plt.ylabel('Energy / z(t)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('energy_z_tsp.png', dpi=300)
plt.show()

# ============================================================
# РЕЗУЛЬТАТЫ
# ============================================================

print("\n--- Результаты TCNN (10-city TSP) ---")
print(f"Глобальный минимум: {global_min} ({100*global_min/num_trials:.1f}%)")
print(f"Локальные минимумы: {local_min} ({100*local_min/num_trials:.1f}%)")
print(f"Недопустимые решения: {invalid} ({100*invalid/num_trials:.1f}%)")

print("\nОжидание по статье [1] (beta=0.005):")
print("Глобум ≈ 99.9%, локальные ≈ 0.1%, недопустимые 0%")
